In [23]:
import os
import random
import torch
from ultralytics import YOLO
from PIL import Image
import torchvision.transforms as T
import torchvision.transforms.functional as F
from io import BytesIO

# Caminhos
SOURCE_DIR = "/Users/pedrofs/Library/CloudStorage/OneDrive-ISCTE-IUL/Mestrado/2ªSem/APVC/Projeto/dataset/yolo-kit-dataset"
OUTPUT_DIR = "/Users/pedrofs/Library/CloudStorage/OneDrive-ISCTE-IUL/Mestrado/2ªSem/APVC/Projeto/dataset/cnn-kit-dataset-final"
YOLO_MODEL_PATH = "runs_yolo/logo_detector/weights/best.pt"

In [24]:
# YOLO carregado
model = YOLO(YOLO_MODEL_PATH)

# Configurações
conf_threshold = 0.4
max_attempts = 50
device = "mps" if torch.backends.mps.is_available() else "cpu"

# Transforms de data augmentation
augmentations = T.Compose([
    T.RandomHorizontalFlip(),
    T.RandomRotation(degrees=15),
    T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    T.RandomResizedCrop(size=(640, 640), scale=(0.8, 1.0))
])

In [25]:
import tempfile

def extract_logo_from_augmented(image, save_base_path, liga, clube):
    for i in range(max_attempts):
        augmented = augmentations(image)

        # Guardar temporariamente imagem aumentada
        with tempfile.NamedTemporaryFile(suffix=".jpg", delete=False) as tmpfile:
            augmented.save(tmpfile.name)
            tmp_path = tmpfile.name

        # YOLO sobre imagem guardada
        results = model.predict(tmp_path, conf=conf_threshold, verbose=False)
        boxes = results[0].boxes.xyxy

        if boxes is not None and len(boxes) > 0:
            for j, box in enumerate(boxes):
                x1, y1, x2, y2 = map(int, box[:4])
                logo_crop = augmented.crop((x1, y1, x2, y2)).resize((224, 224))
                save_dir = os.path.join(save_base_path, liga, clube)
                os.makedirs(save_dir, exist_ok=True)
                logo_crop.save(os.path.join(save_dir, f"{clube}_{i}_{j}.jpg"))
                print(f"Logo guardado: {liga}/{clube}")
            os.remove(tmp_path)  # limpar imagem temporária
            return

        os.remove(tmp_path)  # limpar imagem temporária se não usámos
    print(f" Falha: não encontrado logo para {liga}/{clube}")

In [ ]:
for liga in sorted(os.listdir(SOURCE_DIR)):
    liga_path = os.path.join(SOURCE_DIR, liga)
    if not os.path.isdir(liga_path) or liga.startswith("."):
        continue

    for clube in sorted(os.listdir(liga_path)):
        if clube.startswith("."):
            continue

        output_clube_path = os.path.join(OUTPUT_DIR, liga, clube)
        if os.path.exists(output_clube_path):
            continue  # já processado

        clube_path = os.path.join(liga_path, clube)
        if not os.path.isdir(clube_path):
            continue

        for filename in os.listdir(clube_path):
            if not filename.lower().endswith((".jpg", ".jpeg", ".png")):
                continue

            image_path = os.path.join(clube_path, filename)
            try:
                img = Image.open(image_path).convert("RGB")
                extract_logo_from_augmented(img, OUTPUT_DIR, liga, clube)
            except Exception as e:
                print(f" Erro com {filename}: {e}")